In [2]:
import boto3
import numpy as np
import tensorflow as tf
import os
#import tensorflow_datasets as tfds
import sagemaker
from sagemaker import get_execution_role
import urllib.request
import re
import sys
import math
import json 
from multiprocessing.dummy import Pool as ThreadPool
import glob
import cv2
role = get_execution_role

In [2]:
bucket_name = 'deeplens-imageclass'
session = boto3.Session()
s3 = session.resource('s3')
my_bucket = s3.Bucket(bucket_name)
region = "us-east-1"

In [3]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:latest'}
training_image = containers[session.region_name]

In [65]:
#ds = tfds.load('celeb_a')

In [76]:
def detect_attributes(bucket, key):
    rekognition = session.client('rekognition', region)
    response = rekognition.detect_faces(
            Image = {
                "S3Object": {
                    "Bucket": bucket,
                    "Name": key
                }
            },
            Attributes = [
                "ALL"
            ]
    )
    return response['FaceDetails']

In [81]:
def detect_all_images(image):
    key = image.key
    filename = key.split("/")[2]
    filename = filename.split(".")[0]
    filepath = "data/imgdata-responses/{}.json".format(filename)
    if os.path.isfile(filepath):
        print(key, "processed image")
    else:
        for category in detect_attributes(bucket_name, key):
            with open(filepath, 'w') as outfile:
                json.dump(category, outfile)

In [ ]:
array_images = my_bucket.objects.filter(Prefix='imgdata-raw')
for image in array_images:
    my_key = image.key
    end_my_key = my_key[-4:]
    if end_my_key == ".jpg":
        data_key = my_key.split("/")[2]
        file_pos = "data/" + data_key
        bucket.download_file(my_key, file_pos)
    else:
        continue

In [6]:
pool = ThreadPool(4)
my_map = pool.map(detect_all_images, array_images)

In [90]:
total = 0
for image in array_images:
    my_key = image.key
    end_key = my_key[-4:]
    if end_key == ".jpg":
        total += 1
total

13233

In [105]:
def find_face_border(f_cascade, colored_image):
    scale_factor = 1.1
    copy_image = np.copy(colored_image)
    gray_image = cv2.cvtColor(copy_image, cv2.COLOR_BGR2GRAY)
    faces = f_cascade.detectMultiScale(gray_image, scale_factor, minNeighbors = 5)
    num_faces = len(faces)
    if num_faces == 1:
        (x, y, width, height) = faces[0]
        rec_face = copy_image[x : x + width, y : y + height]
        return rec_face
    else:
        return []

In [112]:
def resize_images():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
    all_images = glob.glob('data/imgdata-raw/*.jpg')
    for image_file in all_images:
        my_image = cv2.imread(image_file)
        rename_image_file = image_file.replace('imgdata-raw', 'imgdata-resize')
        rec_my_image = find_face_border(face_cascade, my_image)
        num_rec_my_image = len(rec_my_image)
        if num_rec_my_image > 0:
            resize_my_image = cv2.resize(rec_my_image, (224, 224))
            cv2.imwrite(rename_image_file, resize_my_image)

In [17]:
def find_emotions():
    train_file = open('training-file.lst', 'w')
    test_file = open('test-file.lst', 'w')
    response_counter = 0
    all_responses = glob.glob('data/imgdata-responses/*.json')
    dict_emotion = {
    
    }
    for filename in all_responses:
        response_counter = response_counter + 1
        spec_file = filename.split('/')[2]
        spec_file = spec_file.split('.')[0]
        image_spec_file = 'data/imgdata-resize/{}.jpg'.format(spec_file)
        if os.path.isfile(image_spec_file):
            data = json.load(open(filename))
            # smile and emotions for later use, for now just single emotion output
            smile = False 
            emotions = []
            emotion = 'None'
            for i in range(7):
                emotions.append(0)
            if data['Smile']['Value'] or data['Smile']['Confidence'] < 89:
                smile = True
            emotion_data = data['Emotions']
            for response_emotion in emotion_data:
                emotion_type = response_emotion['Type']
                emotion_confidence = response_emotion['Confidence']
                if emotion_confidence > 70:
                    if emotion_type == 'HAPPY':
                        emotions[0] = 1 
                        continue
                    elif emotion_type == 'CALM':
                        emotions[1] = 1
                        continue
                    elif emotion_type == 'ANGRY':
                        emotions[2] = 1
                        continue
                    elif emotion_type == 'SAD':
                        emotions[3] = 1
                        continue
                    elif emotion_type == 'SUPRISED':
                        emotions[4] = 1
                        continue
                    elif emotion_type == 'CONFUSED':
                        emotions[5] = 1
                        continue
                    elif emotion_type == 'DISGUSTED':
                        emotions[6] = 1
                        continue
            highest_emotion = -1
            for response_emotion in emotion_data:
                emotion_type = response_emotion['Type']
                emotion_confidence = response_emotion['Confidence']
                if emotion_confidence > highest_emotion:
                    highest_emotion = emotion_confidence
                    emotion = emotion_type
            dict_emotion[image_spec_file] = "Happy" if smile else "Not Happy"
            if response_counter < 9928:
                train_file.write("{} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {} \n".format(
                    emotion, smile, emotions[0], emotions[1], emotions[2], emotions[3], emotions[4], emotions[5], emotions[6], filename))        
            else:
                test_file.write("{} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {} \n".format(
                    emotion, smile, emotions[0], emotions[1], emotions[2], emotions[3], emotions[4], emotions[5], emotions[6], filename))
    with open('emotion-dictionary.json', 'w') as emotion_json_file:
        json.dump(dict_emotion, emotion_json_file)

In [ ]:
resize_images()

In [18]:
find_emotions()

In [3]:
dict_string = 'emotion-dictionary.json'
dict_emotion_file = open(dict_string, 'r')
dict_emotion = json.load(dict_emotion_file)
dict_emotion_list = list(dict_emotion)
happy = 0
not_happy = 0
for emotion in dict_emotion_list:
    if dict_emotion[emotion] == "Happy":
        happy += 1
    else:
        not_happy += 1

In [23]:
my_json_file = 'emotion-dictionary.json'
my_path = os.path.isfile(my_json_file)
if my_path:
    mine = json.load(open(my_json_file))
    print(len(mine))

11646
